In [ ]:
from Methods import*
import pandas as pd

In [ ]:
data = pd.read_excel('G:/ahmed/Final Project ds tools/Egyptian-Cars-Analysis/DataSetScraping3.xlsx')
df = data.copy()

In [ ]:
df_1 = df.copy()
Modify = Modify(df_1)

In [ ]:
df_int =Modify.Extract_int(['Price','Milage'])
df_int[(df_int['Transmission'] != 'أتوماتيك\u200e') & (df_int['Transmission'] != 'مانيوال') & (df_int['Transmission'] !='DSG')]

In [ ]:
df_2 = Modify.fill_na_mileage(df_int,'Year','Milage')

In [ ]:
df_2.loc[df_2['Milage'].isna()]

In [ ]:
df_3 = Modify.fill_zero_mileage(df_2,'Year','Milage')

In [ ]:
df_3.loc[df_3['Milage']==0]

In [ ]:
df_3['Price'] = df_3['Price'].astype('int64')

In [ ]:
df_3.info()

In [ ]:
df_3[(df_3['Transmission'] != 'أتوماتيك\u200e') & (df_3['Transmission'] != 'مانيوال') & (df_3['Transmission'] !='DSG')]

In [ ]:
df_3 = df_3.drop(df_3[df_3['Milage']==0].index)

In [ ]:
df_3.shape

In [ ]:
print(df_3[(df_3['Milage']<10000)&(df_3['Year']<2024)].shape)
Mileage_less_10000 = df_3[(df_3['Milage']<10000)&(df_3['Year']<2024)].index

In [ ]:
df_3[(df_3['Milage']<10000)&(df_3['Year']<2024)]

In [ ]:
df_4 = df_3.copy()

In [ ]:
Group_By_brand_Mileage = df_3.groupby(['Brand','Year'])['Milage'].mean().round().reset_index()
Group_By_brand_Mileage

In [ ]:
for index_1 in Group_By_brand_Mileage.index:
    for index_2 in Mileage_less_10000:
        if (Group_By_brand_Mileage.loc[index_1, 'Brand'] == df_4.loc[index_2, 'Brand']) and (Group_By_brand_Mileage.loc[index_1, 'Year'] == df_4.loc[index_2, 'Year']):
            df_4.loc[index_2, 'Milage'] = Group_By_brand_Mileage.loc[index_1, 'Milage']
    
        

In [ ]:
df_4[df_4['Milage']<1000].sample(10)

In [ ]:
Group_by_Brand_Price = df_4.groupby('Brand')['Price'].mean().round().reset_index()
Group_by_Brand_Price

In [ ]:
Price_than_10000_index = df_3[df_3['Price']<100000].index
df_3[df_3['Price']<100000].shape

In [ ]:
for index_1 in Group_by_Brand_Price.index:
    for index_2 in Price_than_10000_index:
        if (Group_by_Brand_Price.loc[index_1, 'Brand'] == df_4.loc[index_2, 'Brand']):
            df_4.loc[index_2, 'Price'] = Group_by_Brand_Price.loc[index_1, 'Price']

In [ ]:
df_4 = df_4.drop('Options',axis=1)

In [ ]:
df_4[(df_4['Year']==2026)|(df_4['Year']==0)]

In [ ]:
df_4 = df_4.drop(df_4[(df_4['Year']==2026)|(df_4['Year']==0)].index)

In [ ]:
df_4.duplicated().sum()

In [ ]:
df_4 = df_4.drop_duplicates()

In [ ]:
col_brand= df_4['Brand'].apply(lambda x: re.sub(r'\s*\|\|\s*', ' || ', x))


In [ ]:
df_4['Brand'] = df_4['Brand'].apply(lambda x: get_display(arabic_reshaper.reshape(x)))
df_4['Model'] = df_4['Brand'].apply(lambda x: get_display(arabic_reshaper.reshape(x)))
df_4['Transmission'] = df_4['Transmission'].apply(lambda x: get_display(arabic_reshaper.reshape(x)))
df_4['Fuel'] = df_4['Fuel'].apply(lambda x: get_display(arabic_reshaper.reshape(x)))
df_4['Location'] = df_4['Location'].apply(lambda x: get_display(arabic_reshaper.reshape(x)))
df_4['Color'] = df_4['Color'].apply(lambda x: get_display(arabic_reshaper.reshape(x)))

In [ ]:
#the final data set is df_4 and df_5 after removing the outliers

In [ ]:
Visualize = Visualize(df_4)

In [ ]:
Visualize.comparing_Year_with_Milage_price()

In [ ]:
Visualize.heat_map(['Year','Milage','Price'])

In [ ]:
Visualize.Distribution_of_Price_Mileage_Year(df_4)

In [ ]:
df_5 = df_4.copy()
for column in df_5[['Year','Milage','Price']]:
    q1=df_4[column].quantile(0.25)
    q3=df_4[column].quantile(0.75)
    IQR=q3-q1
    df_5 = df_5[~((df_5[column] <= (q1 - 1.5 * IQR)) | (df_5[column] >= (q3 + 1.5 * IQR)))]

In [ ]:
Visualize.Distribution_of_Price_Mileage_Year(df_5)

In [ ]:
df_4[df_4["Transmission"]=='DSG'].index

In [ ]:
Visualize.piechart_Transmission()

In [ ]:
grouping = df_4.groupby('Brand')['Price'].sum().reset_index().sort_values(by='Price',ascending=False).head(10)

In [ ]:
sns.barplot(grouping, x="Brand", y="Price", estimator="sum", errorbar=None)
